# Scaling out numerical computing in Julia
Przemysław Szufel




<a class="anchor" id="toc"></a>
## Table of content

    
1. [Parallelize via Single Instruction Multiple Data (SIMD)](#simd)
2. [Green threading](#green)
3. [Multithreading](#multithreading)
4. [Multi-processing and distributed computing](#multiprocessing)

Before running Jupyter notebook set in Julia number of threads.
This should be done *before* actually running the `notebook()` command.
The number of threads can be also set up in Julia options in Visual Studio code (if this is used to run this notebook).
```
# run this code from Julia console just before starting Jupyter Notebook
ENV["JULIA_NUM_THREADS"]=4
```

In [5]:
println("Number of threads that your Julia is run: ## $(Threads.nthreads())")

Number of threads that your Julia is run: ## 1


In [6]:
]status

Status `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julica_EC_Distributed_GPU\1_ParallelDistributed\Project.toml`
  [1520ce14] AbstractTrees v0.4.5
  [6e4b80f9] BenchmarkTools v1.5.0
  [336ed68f] CSV v0.10.14
  [34f1f09b] ClusterManagers v0.4.6
  [a93c6f00] DataFrames v1.6.1
  [31c24e10] Distributions v0.25.108
  [cd3eb016] HTTP v1.10.8
  [0f8b85d8] JSON3 v1.14.0
  [91a5bcdd] Plots v1.40.4


In [7]:
using BenchmarkTools, Distributed

<a class="anchor" id="simd"></a>
### Parallelize via Single Instruction Multiple Data (SIMD)
---- [Return to table of contents](#toc) ---



In [8]:
function dot1(x, y)
    s = 0.0
    for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

dot1 (generic function with 1 method)

In [9]:
function dot2(x, y)
    s = 0.0
    @simd for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

dot2 (generic function with 1 method)

In [10]:
x = 100*rand(10000)
y = 100*rand(10000);

@btime dot1($x, $y)
@btime dot2($x, $y)

  4.000 μs (0 allocations: 0 bytes)
  764.912 ns (0 allocations: 0 bytes)


2.522111788782171e7

In [11]:
res1 =  dot1(x, y)

2.5221117887821734e7

In [12]:
res2 =  dot2(x, y)

2.522111788782171e7

In [13]:
res1 == res2

false

In [14]:
@show res1 
@show res2

res1 = 2.5221117887821734e7
res2 = 2.522111788782171e7


2.522111788782171e7

In [15]:
res1 ≈ res2

true

<a class="anchor" id="green"></a>
### Green threading 
---- [Return to table of contents](#toc) ---


In [16]:
@time sleep(2)

  2.004744 seconds (43 allocations: 1024 bytes)


In [17]:
@time t = @async sleep(4)

  0.005987 seconds (2.80 k allocations: 201.922 KiB, 99.07% compilation time)


Task (runnable) @0x000001d22f858010

In [18]:
function dojob(i)
    fname = download("https://szufel.pl/pliki/plik.txt")  # this could be external computations with I/O
    i, fname
end

dojob (generic function with 1 method)

In [19]:
result = Vector{Tuple{Int,String}}(undef, 8)

8-element Vector{Tuple{Int64, String}}:
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef

In [20]:
@time for i=1:8
    result[i] = dojob(i)
end
result

  1.362254 seconds (17.80 k allocations: 1.242 MiB, 1.62% compilation time)


8-element Vector{Tuple{Int64, String}}:
 (1, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_rklcXuDePe")
 (2, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_5cl0VaN0Lm")
 (3, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_HetezUnuvQ")
 (4, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_3yx63I3Wb2")
 (5, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_XSZA58rmVM")
 (6, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_varCpWfovs")
 (7, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_tWLcVcnwhG")
 (8, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_F4FaX29e5S")

In [21]:
result = Vector{Tuple{Int,String}}(undef, 8)
@time for i=1:8
   @async result[i] = dojob(i)
end
result

  0.000079 seconds (81 allocations: 7.055 KiB)


8-element Vector{Tuple{Int64, String}}:
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef

In [22]:
result


8-element Vector{Tuple{Int64, String}}:
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef

In [23]:
result = Vector{Tuple{Int,String}}(undef, 8);
@time @sync for i=1:8
   @async result[i] = dojob(i)
end
result

  3.099014 seconds (6.25 k allocations: 347.023 KiB, 0.56% compilation time)


8-element Vector{Tuple{Int64, String}}:
 (1, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_fS7W3yTgN0")
 (2, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_JCNgZcLcX2")
 (3, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_7cPynilaNe")
 (4, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_tSpoDsxsdQ")
 (5, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_nyZubeDInm")
 (6, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_xO1EDgZi70")
 (7, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_zqFMzUnotC")
 (8, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_dOfcnmVolQ")

In [24]:
asyncmap(dojob, 1:8)

8-element Vector{Tuple{Int64, String}}:
 (1, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_FsV2F6RY1c")
 (2, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_POfmVgrWZO")
 (3, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_daZ2J0vUt2")
 (4, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_z2tQ54rkzA")
 (5, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_tozeJwXq5I")
 (6, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_BcdmJyJMZo")
 (7, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_t0rWbwhCPe")
 (8, "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_RYf8L6fOpa")

In [25]:
?asyncmap

search: asyncmap asyncmap!



```
asyncmap(f, c...; ntasks=0, batch_size=nothing)
```

Uses multiple concurrent tasks to map `f` over a collection (or multiple equal length collections). For multiple collection arguments, `f` is applied elementwise.

`ntasks` specifies the number of tasks to run concurrently. Depending on the length of the collections, if `ntasks` is unspecified, up to 100 tasks will be used for concurrent mapping.

`ntasks` can also be specified as a zero-arg function. In this case, the number of tasks to run in parallel is checked before processing every element and a new task started if the value of `ntasks_func` is greater than the current number of tasks.

If `batch_size` is specified, the collection is processed in batch mode. `f` must then be a function that must accept a `Vector` of argument tuples and must return a vector of results. The input vector will have a length of `batch_size` or less.

The following examples highlight execution in different tasks by returning the `objectid` of the tasks in which the mapping function is executed.

First, with `ntasks` undefined, each element is processed in a different task.

```
julia> tskoid() = objectid(current_task());

julia> asyncmap(x->tskoid(), 1:5)
5-element Array{UInt64,1}:
 0x6e15e66c75c75853
 0x440f8819a1baa682
 0x9fb3eeadd0c83985
 0xebd3e35fe90d4050
 0x29efc93edce2b961

julia> length(unique(asyncmap(x->tskoid(), 1:5)))
5
```

With `ntasks=2` all elements are processed in 2 tasks.

```
julia> asyncmap(x->tskoid(), 1:5; ntasks=2)
5-element Array{UInt64,1}:
 0x027ab1680df7ae94
 0xa23d2f80cd7cf157
 0x027ab1680df7ae94
 0xa23d2f80cd7cf157
 0x027ab1680df7ae94

julia> length(unique(asyncmap(x->tskoid(), 1:5; ntasks=2)))
2
```

With `batch_size` defined, the mapping function needs to be changed to accept an array of argument tuples and return an array of results. `map` is used in the modified mapping function to achieve this.

```
julia> batch_func(input) = map(x->string("args_tuple: ", x, ", element_val: ", x[1], ", task: ", tskoid()), input)
batch_func (generic function with 1 method)

julia> asyncmap(batch_func, 1:5; ntasks=2, batch_size=2)
5-element Array{String,1}:
 "args_tuple: (1,), element_val: 1, task: 9118321258196414413"
 "args_tuple: (2,), element_val: 2, task: 4904288162898683522"
 "args_tuple: (3,), element_val: 3, task: 9118321258196414413"
 "args_tuple: (4,), element_val: 4, task: 4904288162898683522"
 "args_tuple: (5,), element_val: 5, task: 9118321258196414413"
```


#### Programming a simple web server
You should be able to connect using the address <a href="http://localhost:9992/3+4" target="about:blank">http://localhost:9992/3+4</a>

To stop web server click <a href="http://localhost:9992/stopme" target="about:blank">http://localhost:9992/stopme</a>

In [26]:
using Sockets
println("Starting the web server...")
server = Sockets.listen(9992)

Starting the web server...


Sockets.TCPServer(Base.Libc.WindowsRawSocket(0x0000000000000970) active)

In [27]:
@async begin
    contt = Ref(true)
    while contt[]
        sock = Sockets.accept(server)
        @async begin
            data = readline(sock)
            print("Got request:\n", data, "\n")
            cmd = split(data, " ")[2][2:end]
            println(sock, "\nHTTP/1.1 200 OK\nContent-Type: text/html\n")
            contt[] = contt[] && (!occursin("stopme", data))
            if contt[]
                 println(sock, string("<html><body>", cmd, "=", 
                     eval(Meta.parse(cmd)), "</body></html>"))
            else
                println(sock,"<html><body>stopping</body></html>")
            end
            close(sock)
        end
    end
    println("Handling requests stopped")
end

Task (runnable) @0x000001d269567270

<a class="anchor" id="multithreading"></a>
### Multithreading
---- [Return to table of contents](#toc) ---


In [1]:
Threads.nthreads()

8

In [2]:
function ssum(x)
    r, c = size(x)
    y = zeros(c)
    for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end

ssum (generic function with 1 method)

In [3]:
function tsum(x)
    r, c = size(x)
    y = zeros(c)
    Threads.@threads for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end


tsum (generic function with 1 method)

In [31]:
x = rand(1000,10000);

In [32]:
@time ssum(x)
@time ssum(x);

  0.012313 seconds (7.23 k allocations: 575.148 KiB, 55.73% compilation time)
  0.007010 seconds (2 allocations: 78.172 KiB)


In [33]:
@time tsum(x)
@time tsum(x);

  0.055239 seconds (41.80 k allocations: 2.940 MiB, 90.32% compilation time)
  0.005430 seconds (10 allocations: 79.078 KiB)


#### Locking mechanism for threads

In [4]:
function f_bad()
    x = 0
    Threads.@threads for i in 1:10^6
        x += 1
    end
    return x
end


f_bad (generic function with 1 method)

In [5]:
f_bad()

183975

In [8]:
using BenchmarkTools
function f_add()
    x = 0 
    for i in 1:10^6
        x += 1
    end
    x
end
@btime f_add() 

  5.700 ns (0 allocations: 0 bytes)


1000000

In [9]:
function f_atomic()
    x = Threads.Atomic{Int}(0)
    Threads.@threads for i in 1:10^6
        Threads.atomic_add!(x, 1)
    end
    return x[]
end
f_atomic()

1000000

In [10]:
function f_easy()
    result = zeros(Int, Threads.nthreads())
    Threads.@threads for i in 1:10^6
        result[Threads.threadid()] += 1
    end
    return sum(result)
end

f_easy (generic function with 1 method)

In [20]:
function f_chunks()
    chunks = Iterators.partition(1:1000_000, 1000_000 ÷ Threads.nthreads())
    tasks = map(chunks) do chunk
        Threads.@spawn begin
            s = 0
            for i in chunk
                s += 1
            end
            s
        end
    end
    chunk_sums = fetch.(tasks)
    return sum(chunk_sums)
end

f_chunks (generic function with 2 methods)

In [11]:

function f_reentrant()
    l = ReentrantLock()
    x = 0
    Threads.@threads for i in 1:10^6
        Threads.lock(l) do
            x += 1
        end
    end
    return x
end


f_reentrant (generic function with 1 method)

In [21]:
using DataFrames
stats = DataFrame()
for f in [f_bad, f_atomic, f_easy, f_chunks, f_reentrant]
    for i = 1:2
        value, elapsedtime  = @timed f()
        push!(stats, (f=string(f),i=i, value=value, timems=elapsedtime*1000))
    end
end
println(stats)


10×4 DataFrame
 Row │ f            i      value    timems   
     │ String       Int64  Int64    Float64  
─────┼───────────────────────────────────────
   1 │ f_bad            1   346812   85.8522
   2 │ f_bad            2   821294  106.447
   3 │ f_atomic         1  1000000   27.5742
   4 │ f_atomic         2  1000000   17.9079
   5 │ f_easy           1  1000000    6.4066
   6 │ f_easy           2  1000000    4.2852
   7 │ f_chunks         1  1000000  171.952
   8 │ f_chunks         2  1000000    4.8388
   9 │ f_reentrant      1  1000000  766.723
  10 │ f_reentrant      2  1000000  878.867


<a class="anchor" id="multiprocessing"></a>
### Multi-processing and distributed computing
---- [Return to table of contents](#toc) ---


In [41]:
using Distributed

This code adds 4 workers (and avoids adding more)

In [42]:
addprocs(max(0, 5-nprocs()));

In [43]:
workers()

4-element Vector{Int64}:
 2
 3
 4
 5

In [44]:
function s_rand()
    n = 10^4
    x = 0.0
    for i in 1:n
        x += sum(rand(10^4))
    end
    x / n
end
 
@time s_rand()
@time s_rand()


  0.302316 seconds (20.00 k allocations: 763.397 MiB, 18.63% gc time)
  0.320021 seconds (20.00 k allocations: 763.397 MiB, 12.46% gc time)


5000.1002833351995

In [45]:
using Distributed
 
function p_rand()
    n = 10^4
    x = @distributed (+) for i in 1:n
        # the last line will be aggregated
        sum(rand(10^4))
    end
    x / n
end

@time p_rand()
@time p_rand()


  1.191995 seconds (465.08 k allocations: 31.229 MiB, 0.99% gc time, 35.13% compilation time)
  0.126764 seconds (424 allocations: 24.047 KiB)


4999.982365060395

In [46]:
workers()'

1×4 adjoint(::Vector{Int64}) with eltype Int64:
 2  3  4  5

In [47]:
fetch(@spawnat 3 4+3)

7

In [48]:
function myf() 
    println("I am on worker ", myid())
    rand()
end
myf()

I am on worker 1


0.26320790413310036

In [49]:
a = nothing
try 
    fetch(@spawnat 4 myf())
catch e
    println(e)
end

RemoteException(4, CapturedException(UndefVarError(Symbol("#myf")), Any[(deserialize_datatype at Serialization.jl:1399, 1), (handle_deserialize at Serialization.jl:867, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_global_from_main at clusterserialize.jl:160, 1), (#5 at clusterserialize.jl:72 [inlined], 1), (foreach at abstractarray.jl:3097, 1), (deserialize at clusterserialize.jl:72, 1), (handle_deserialize at Serialization.jl:960, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:871, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_msg at messages.jl:87, 1), (#invokelatest#2 at essentials.jl:892 [inlined], 1), (invokelatest at essentials.jl:889 [inlined], 1), (message_handler_loop at process_messages.jl:176, 1), (process_tcp_s

In [50]:
@everywhere function myf() 
    println("I am on worker ", myid())
    rand()
end
fetch(@spawnat 4 myf())

      From worker 4:	I am on worker 4


0.43466090764226784

#### A typical pattern for setting an intial state across workers

In [54]:
using Distributed
@everywhere using Pkg
# This assumes that the package is in the same folder as the running code
@everywhere Pkg.activate(".")
@everywhere using Distributed, Random, DataFrames

@everywhere function calc(x, y)
    2x + y
end

@everywhere function init_worker()    
   Random.seed!(myid())
    # reading initial data from files or other actions
end

@sync for wid in workers()
    @async fetch(@spawnat wid init_worker())
end


      From worker 2:	  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julica_EC_Distributed_GPU\1_ParallelDistributed`
      From worker 5:	  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julica_EC_Distributed_GPU\1_ParallelDistributed`
      From worker 4:	  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julica_EC_Distributed_GPU\1_ParallelDistributed`
      From worker 3:	  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julica_EC_Distributed_GPU\1_ParallelDistributed`


  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julica_EC_Distributed_GPU\1_ParallelDistributed`


Typically results are collected to a `DataFrame`

In [55]:
data = @distributed (append!) for (i, j) = vec(collect(Iterators.product(1:4, 1:3)))
    a = rand(1:499)
    b = rand(1:9)*1000
    c = calc(a, b)
    DataFrame(;i,j,a,b,c,procid = myid())
end

Row,i,j,a,b,c,procid
,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,467,6000,6934,2
2,2,1,14,2000,2028,2
3,3,1,395,9000,9790,2
4,4,1,421,9000,9842,3
5,1,2,265,8000,8530,3
6,2,2,207,3000,3414,3
7,3,2,30,7000,7060,4
8,4,2,24,6000,6048,4
9,1,3,181,3000,3362,4


In [74]:
pmap(i-> (;i, procid=myid(), c= calc(i,i) ), 1:10) |> DataFrame

Row,i,procid,c
,Int64,Int64,Int64
1,1,4,3
2,2,3,6
3,3,5,9
4,4,2,12
5,5,5,15
6,6,4,18
7,7,5,21
8,8,5,24
9,9,4,27
